In [3]:
import keras
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os
import json
import pandas as pd

/home/yajingyang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Dogs and cats dataset

In [4]:
import random
img_dir = '/home/yajingyang/Downloads/PetImages/'

def generate_noisy_binary_label(y_train, N_ANNOT, num_labels):
    noise_label_list = []
#     pos_acc_list = [ 0.7, 0.8, 0.5, 0.2, 0.9]
#     neg_acc_list = [ 0.6, 0.9, 0.5, 0.9, 0.9]
    pos_acc_list = [0.65, 0.65, 0.65, 0.65, 0.65]
    neg_acc_list = [0.65, 0.65, 0.65, 0.65, 0.65]
    for a in range(N_ANNOT):
        noisy_label = y_train.copy()
        temp = y_train.copy()
        noisy_label[np.where(temp)] = np.random.binomial(1,pos_acc_list[a], temp[np.where(temp)].shape[0])
        noisy_label[np.where(temp==0)] = np.random.binomial(1, (1-neg_acc_list[a]), temp[np.where(temp==0)].shape[0])
        noisy_label = np.transpose(noisy_label)
        noise_label_list.append(noisy_label)
    noise_label_arr = np.array(noise_label_list)
    noise_label_arr = np.transpose(noise_label_arr)
#     for a in range(N_ANNOT):
#         noise_label_arr[:,a][np.random.choice(noise_label_arr.shape[0], 
#                                               noise_label_arr.shape[0] - num_labels, replace=False)] = -1
#     for i in range(noise_label_arr.shape[0]):
#         if all(noise_label_arr[i,:] == -1):
#             a = np.random.choice(N_ANNOT, 1)
#             noise_label_arr[i,a] = noise_label_arr[:,a][np.where(noise_label_arr[:, a]!=-1)][0]
#             noise_label_arr[:,a][np.where(noise_label_arr[:, a]!=-1)][0] = -1
    return noise_label_arr

def get_mv_binary_label(noise_label_arr):
    mv_list = []
    for i in range(noise_label_arr.shape[0]):
        if all(noise_label_arr[i,:][np.where(noise_label_arr[i, :]!=-1)] == 1):
            mv = 1
        elif np.sum(noise_label_arr[i,:][np.where(noise_label_arr[i, :]!=-1)]) == 0:
            mv = 0
        elif noise_label_arr[i,:][np.where(noise_label_arr[i, :]==1)].shape[0] == \
            noise_label_arr[i,:][np.where(noise_label_arr[i, :]==0)].shape[0]:
            mv = np.random.binomial(1,0.5)
        else:
            mv = np.bincount(noise_label_arr[i,:][np.where(noise_label_arr[i, :]!=-1)]).argmax()
        mv_list.append(mv)
    mv_arr = np.array(mv_list)
    return mv_arr

def randomize(arr_list):
    # Generate the permutation index array.
    permutation = np.random.permutation(arr_list[0].shape[0])
    # Shuffle the arrays by giving the permutation in the square brackets.
    rand_arr_list = []
    for arr in arr_list: 
        rand_arr_list.append(arr[permutation])
    return rand_arr_list

def process_cat_dog_dataset(img_dir, sample=1):
    fn_list = [fn for fn in os.listdir(img_dir) if fn.endswith('jpg')]
    N_ANNOT = 5

#     if sample==1:
#         save_dir = img_dir
#     else:
    save_dir = img_dir+'/annot_5_acc_0.65/'
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
    import random
    random.seed(45) 
    random.shuffle(fn_list)
    
    model = VGG16(weights='imagenet', include_top=False)
    train_fn_list = fn_list[:round(len(fn_list)*sample*0.5)]
    val_fn_list = fn_list[round(len(fn_list)*sample*0.5):round(len(fn_list)*sample*0.75)]
    test_fn_list = fn_list[round(len(fn_list)*sample*0.75):round(len(fn_list)*sample)]
    print(len(train_fn_list), len(val_fn_list), len(test_fn_list))

    for split in [train_fn_list, val_fn_list, test_fn_list]:
        gt_list = []
        feat_list = []

        for i, file in enumerate(split):
            img_path = img_dir + file
            img = image.load_img(img_path, target_size=(150, 150))
            img = image.img_to_array(img)

            x = np.expand_dims(img, axis=0)
            x = preprocess_input(x)
            features = model.predict(x)
            feat_list.append(features)
            gt_list.append(int(file[0]))
        
        output_gt = np.array(gt_list)
        feat_array = np.array(feat_list)
        feat_array = np.squeeze(feat_array)
        print(feat_array.shape)
        print(output_gt.shape)

        if split == train_fn_list:
            output_label = generate_noisy_binary_label(output_gt, N_ANNOT, num_labels = 500)
            output_mv = get_mv_binary_label(output_label)
            y_labels_mix_list = []

            for pct in [1, 2, 3, 4, 5]:
                x_sample = feat_array[:round(len(train_fn_list)* pct / 100)]
                y_sample = output_gt[:round(len(train_fn_list)* pct / 100)]
                x_sample, y_sample = randomize([x_sample, y_sample])
                np.save(save_dir+'data_train_vgg16_%dpct.npy'%pct, x_sample)
                np.save(save_dir+'labels_train_%dpct.npy'%pct, y_sample)
            
                y_labels_mix = output_label.copy()
                y_labels_mix[:round(y_labels_mix.shape[0]*pct/100)] \
                        = np.reshape(np.repeat(output_gt[:round(y_labels_mix.shape[0]*pct/100)], N_ANNOT, axis=0), 
                                     [round(y_labels_mix.shape[0]*pct/100), N_ANNOT])
                y_labels_mix_list.append(y_labels_mix)
            print(feat_array.shape, output_gt.shape, output_mv.shape, output_label.shape)
            arr_list = [feat_array, output_gt, output_mv, output_label]
            arr_list.extend(y_labels_mix_list)
                
            feat_array, output_gt, output_mv, output_label, y_labels_mix_list[0], y_labels_mix_list[1], \
                    y_labels_mix_list[2],y_labels_mix_list[3],y_labels_mix_list[4]= \
                    randomize(arr_list)

            for y_labels_mix, pct in zip(y_labels_mix_list, [1,2,3,4,5]):
                np.save(save_dir+'data_train_mix_%dpct.npy'%pct, y_labels_mix)
            np.save(save_dir+'data_train_vgg16.npy', feat_array)
            np.save(save_dir+'labels_train.npy', output_gt)
            np.save(save_dir+'answers.npy', output_label)
            np.save(save_dir+'labels_train_mv.npy', output_mv)
        elif split == val_fn_list:
            np.save(save_dir+'data_valid_vgg16.npy', feat_array)
            np.save(save_dir+'labels_valid.npy', output_gt)
        else:
            np.save(save_dir+'data_test_vgg16.npy', feat_array)
            np.save(save_dir+'labels_test.npy', output_gt)
    return 

In [5]:
DATA_PATH = "/home/yajingyang/Downloads/PetImages/"
process_cat_dog_dataset(DATA_PATH)

12499 6249 6250


/home/yajingyang/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 209715200 bytes but only got 0. Skipping tag 48
  " Skipping tag %s" % (size, len(data), tag))
/home/yajingyang/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 52428800 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/yajingyang/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 6468665344 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/yajingyang/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1050744 bytes but only got 7027. Skipping tag 48
  " Skipping tag %s" % (size, len(data), tag))
/home/yajingyang/anaconda3/lib/python3.6/site-pac

/home/yajingyang/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 262146 bytes but only got 0. Skipping tag 2
  " Skipping tag %s" % (size, len(data), tag))
/home/yajingyang/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 262151 bytes but only got 0. Skipping tag 56
  " Skipping tag %s" % (size, len(data), tag))
/home/yajingyang/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:756: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 0. 
  warnings.warn(str(msg))
/home/yajingyang/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 6553600 bytes but only got 0. Skipping tag 49
  " Skipping tag %s" % (size, len(data), tag))
/home/yajingyang/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corr

(12499, 4, 4, 512)
(12499,)
(12499, 4, 4, 512) (12499,) (12499,) (12499, 5)
(6249, 4, 4, 512)
(6249,)


/home/yajingyang/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag))
/home/yajingyang/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag))
/home/yajingyang/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag))
/home/yajingyang/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag))
/home/yajingyang/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.

(6250, 4, 4, 512)
(6250,)


In [46]:
save_dir = DATA_PATH
y_labels_mix = []
for pct in [1,2,3,4,5]:
    y_labels_mix.append(np.load(save_dir+'data_train_mix_%dpct.npy'%pct))
x_train =  np.load(save_dir+'data_train_vgg16.npy')
y_train = np.load(save_dir+'labels_train.npy')
ans = np.load(save_dir+'answers.npy')
mv = np.load(save_dir+'labels_train_mv.npy')
x_valid = np.load(save_dir+'data_valid_vgg16.npy')
y_valid = np.load(save_dir+'labels_valid.npy')
x_test = np.load(save_dir+'data_test_vgg16.npy')
y_test = np.load(save_dir+'labels_test.npy')

In [17]:
y_test.shape

(250,)

In [47]:
x_train

array([[[[143., 107.,  85.],
         [138., 102.,  80.],
         [138., 103.,  81.],
         ...,
         [175., 158., 148.],
         [153., 135., 113.],
         [189., 177., 163.]],

        [[139., 102.,  83.],
         [135., 100.,  78.],
         [134.,  99.,  77.],
         ...,
         [170., 153., 143.],
         [159., 141., 121.],
         [182., 170., 158.]],

        [[145., 113.,  98.],
         [133., 102.,  84.],
         [137., 105.,  84.],
         ...,
         [162., 145., 135.],
         [153., 134., 117.],
         [188., 178., 168.]],

        ...,

        [[106., 185., 166.],
         [104., 187., 161.],
         [108., 192., 166.],
         ...,
         [253., 242., 236.],
         [187., 171., 158.],
         [193., 192., 197.]],

        [[108., 187., 168.],
         [103., 186., 160.],
         [104., 188., 162.],
         ...,
         [255., 246., 240.],
         [158., 142., 129.],
         [232., 231., 236.]],

        [[107., 186., 167.],
       